## 사전 준비

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

import tensorflow as tf
from transformers import TFElectraModel

c:\project_huiyeol\nlp\project\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tf.random.set_seed(42)
np.random.seed(42)
np.set_printoptions(precision=6, suppress=True)

In [3]:
CLASS_NUMBER = 3
NEG_CLASS_NUMBER = 5
MAX_LEN = 40
MAX_WORD = 20
BERT_CKPT = './data_out/'
DATA_IN_PATH = '../metadata/'
DATA_OUT_PATH = './data_out/'

### 토크나이저, 라벨인코더 로드

In [4]:
# Load Electra Tokenizer
with open(DATA_OUT_PATH+'tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [5]:
# Load 3classes(neg, neut, pos) LabelEncoder
with open(DATA_OUT_PATH+'3classEncoder.pickle', 'rb') as handle:
    le = pickle.load(handle)

In [6]:
# Load 5classes(angry, dislike, fear, sad, surprise) LabelEncoder
with open(DATA_OUT_PATH+'negClassEncoder.pickle', 'rb') as handle:
    neg_le = pickle.load(handle)

In [7]:
# Fuctiong for Tokenizing Sentence
def electra_tokenizer(sent, MAX_LEN):
    encoded_dict = loaded_tokenizer.encode_plus(
        text=sent,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )

    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']

    return input_id, attention_mask, token_type_id

### 모델 정의

In [8]:
class TFElectraClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super().__init__()

        self.bert = TFElectraModel.from_pretrained(model_name, cache_dir=dir_path, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.layers.Dense(num_class, name='classifier', activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range))

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs[0]
        last_hidden_state = self.flatten(last_hidden_state)
        last_hidden_state = self.dropout(last_hidden_state, training=training)
        logits = self.classifier(last_hidden_state)

        return logits

In [9]:
# ElectraClassifier for classifying 3classes(neg, neut, pos)
cls_model = TFElectraClassifier(model_name='monologg/koelectra-base-v3-discriminator', dir_path=os.path.join(BERT_CKPT, 'model'), num_class=CLASS_NUMBER)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

In [10]:
# ElectraClassifier for classifying 5classes(angry, dislike, fear, sad, surprise)
neg_model = TFElectraClassifier(model_name='monologg/koelectra-base-v3-discriminator', dir_path=os.path.join(BERT_CKPT, 'model'), num_class=NEG_CLASS_NUMBER)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

In [15]:
# For Loading Weights
a, b, c = electra_tokenizer('안녕하세요', MAX_LEN)
cls_model.call((np.array(a).reshape(1,-1), np.array(b).reshape(1,-1), np.array(c).reshape(1,-1)))
cls_model.built = True
cls_model.load_weights(DATA_OUT_PATH+'tf2_electra_plutchik_hs_6.h5')

In [21]:
# For Loading Weights
a, b, c = electra_tokenizer('안녕하세요', MAX_LEN)
neg_model.call((np.array(a).reshape(1,-1), np.array(b).reshape(1,-1), np.array(c).reshape(1,-1)))
neg_model.built = True
neg_model.load_weights(DATA_OUT_PATH+'tf2_electra_plutchik_hs_10.h5')

In [16]:
# Compile
optimizer = tf.keras.optimizers.Adam(3e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

### 예측 준비

In [17]:
# Function for Preprocessing 3classes classification(neg, neut, pos)
def preprocessing(x):
    temp = x.split('\n')
    temp = ' '.join(temp)
    return temp

def slicing(x):
    x = x.split()
    res = []
    for i in range(0, len(x)+1, MAX_WORD):
        if len(x)-i-MAX_WORD < MAX_WORD//2:
            temp = x[i:]
            res.append(temp)
            break
        temp = x[i:i+MAX_WORD]
        res.append(temp)
    return res

def predict(lyrics):
    temp = []
    lyrics = preprocessing(lyrics)
    lyrics = slicing(lyrics)
    for lyric in lyrics:
        try:
            txt = ' '.join(lyric)
            temp.append(txt)
        except:
            continue

    input_ids = []
    attention_masks = []
    token_type_ids = []

    for lyric in temp:
        try:
            input_id, attention_mask, token_type_id = electra_tokenizer(lyric, MAX_LEN)
            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)

        except Exception as e:
            print(e)
            pass
    prediction = cls_model.predict(input_ids)

    return prediction

def transform(array):
    result = le.inverse_transform([np.argmax(array)])
    return result

In [23]:
# Function for Preprocessing 5classes classification(angry, dislike, fear, sad, surprise)
def neg_predict(lyrics):
    temp = []
    lyrics = preprocessing(lyrics)
    lyrics = slicing(lyrics)
    for lyric in lyrics:
        try:
            txt = ' '.join(lyric)
            temp.append(txt)
        except:
            continue

    input_ids = []
    attention_masks = []
    token_type_ids = []

    for lyric in temp:
        try:
            input_id, attention_mask, token_type_id = electra_tokenizer(lyric, MAX_LEN)
            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)

        except Exception as e:
            print(e)
            pass
    prediction = neg_model.predict(input_ids)

    return prediction

def neg_transform(array):
    result = neg_le.inverse_transform([np.argmax(array)])
    return result

## 예측

In [24]:
lyrics = pd.read_csv(DATA_IN_PATH+'db에넣을노래들.csv')
data = lyrics[['SONG_ID', 'SONG_TITLE', 'LYRICS']]

In [25]:
song_ids = lyrics['SONG_ID']

In [26]:
pred = dict()

In [27]:
for id in song_ids:
    lyric = data[data['SONG_ID']==id]['LYRICS'].values[0]
    prediction = predict(lyric)
    score = sum(prediction)/len(prediction)
    senti = transform(score)
    pred[id] = (score, senti)

1/1 [==============================] - 0s 126ms/step


In [29]:
data['score'] = ''
data['senti'] = ''

c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
for id in song_ids:
    sc, se = pred[id]
    data.loc[data[data['SONG_ID']==id].index, 'score'] = str(sc)
    data.loc[data[data['SONG_ID']==id].index, 'senti'] = se

c:\project_huiyeol\nlp\project\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\project_huiyeol\nlp\project\venv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [33]:
neg_data = data[data['senti']=='neg']
pos_data = data[data['senti']=='pos']
neg_ids = neg_data['SONG_ID']
pos_ids = pos_data['SONG_ID']

In [34]:
pos_pred = dict()
neg_pred = dict()

In [35]:
for id in neg_ids:
    lyric = data[data['SONG_ID']==id]['LYRICS'].values[0]
    prediction = neg_predict(lyric)
    score = sum(prediction)/len(prediction)
    senti = neg_transform(score)
    neg_pred[id] = (score, senti)

1/1 [==============================] - 0s 101ms/step


In [36]:
data['final_score'] = ''
data['final_senti'] = ''
data['total'] = ''

c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [37]:
for key in neg_ids:
    sc, se = neg_pred[key]
    data.loc[data[data['SONG_ID']==key].index, 'final_score'] = str(sc)
    data.loc[data[data['SONG_ID']==key].index, 'final_senti'] = se

c:\project_huiyeol\nlp\project\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\project_huiyeol\nlp\project\venv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [38]:
for key in neg_ids:
    neg_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'final_score'].tolist()[0][1:-1], dtype=float, sep=' ')
    pos_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'score'].tolist()[0][1:-1], dtype=float, sep=' ')
    ratio = np.array(((pos_sc[int(le.transform(['neg'])[0])]+pos_sc[1]*0.5), (pos_sc[int(le.transform(['pos'])[0])]+pos_sc[1]*0.5)))
    rated_sc = np.append(np.max(neg_sc)*(ratio[1]/ratio[0]), neg_sc)
    total_sc = rated_sc/(sum(rated_sc))
    data.loc[data[data['SONG_ID']==key].index, 'total'] = str(total_sc)

In [39]:
for id in pos_ids:
    lyric = data[data['SONG_ID']==id]['LYRICS'].values[0]
    prediction = neg_predict(lyric)
    score = sum(prediction)/len(prediction)
    pos_pred[id] = (score, 'happy')

1/1 [==============================] - 0s 119ms/step


In [40]:
for key in pos_ids:
    sc, se = pos_pred[key]
    data.loc[data[data['SONG_ID']==key].index, 'final_score'] = str(sc)
    data.loc[data[data['SONG_ID']==key].index, 'final_senti'] = se

In [41]:
for key in pos_ids:
    neg_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'final_score'].tolist()[0][1:-1], dtype=float, sep=' ')
    pos_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'score'].tolist()[0][1:-1], dtype=float, sep=' ')
    ratio = np.array(((pos_sc[int(le.transform(['neg'])[0])]+pos_sc[1]*0.5), (pos_sc[int(le.transform(['pos'])[0])]+pos_sc[1]*0.5)))
    rated_sc = np.append(ratio[1], ratio[0]*neg_sc)
    total_sc = rated_sc/(sum(rated_sc))
    data.loc[data[data['SONG_ID']==key].index, 'total'] = str(total_sc)

In [42]:
data = data.drop(data[data['senti']=='neut'].index)

In [44]:
data.to_csv(DATA_IN_PATH+'testLyrics.csv', encoding='utf-8-sig')

## 클래스 정의

In [9]:
class EmotionClassfier():
    def __init__(self,encoder1, encoder2):

        self.model1 = TFElectraClassifier(model_name='monologg/koelectra-base-v3-discriminator', dir_path=os.path.join(BERT_CKPT, 'model'), num_class=CLASS_NUMBER)
        self.model2 = TFElectraClassifier(model_name='monologg/koelectra-base-v3-discriminator', dir_path=os.path.join(BERT_CKPT, 'model'), num_class=NEG_CLASS_NUMBER)
        self.encoder1 = encoder1
        self.encoder2 = encoder2

    def call(self):
        # For Loading Weights
        a, b, c = electra_tokenizer('안녕하세요', MAX_LEN)
        self.model1.call((np.array(a).reshape(1,-1), np.array(b).reshape(1,-1), np.array(c).reshape(1,-1)))
        self.model1.built = True
        self.model1.load_weights(DATA_OUT_PATH+'tf2_electra_plutchik_hs_6.h5')
        # For Loading Weights
        a, b, c = electra_tokenizer('안녕하세요', MAX_LEN)
        self.model2.call((np.array(a).reshape(1,-1), np.array(b).reshape(1,-1), np.array(c).reshape(1,-1)))
        self.model2.built = True
        self.model2.load_weights(DATA_OUT_PATH+'tf2_electra_plutchik_hs_10.h5')

    # Function for Preprocessing 3classes classification(neg, neut, pos)
    def preprocessing(self, lyric):
        temp = lyric.split('\n')
        temp = ' '.join(temp)
        return temp

    def slicing(self, lyric):
        lyric = lyric.split()
        res = []
        for i in range(0, len(lyric)+1, MAX_WORD):
            if len(lyric)-i-MAX_WORD < MAX_WORD//2:
                temp = lyric[i:]
                res.append(temp)
                break
            temp = lyric[i:i+MAX_WORD]
            res.append(temp)
        return res

    def predict3classes(self, lyrics):
        temp = []
        lyrics = self.preprocessing(lyrics)
        lyrics = self.slicing(lyrics)
        for lyric in lyrics:
            try:
                txt = ' '.join(lyric)
                temp.append(txt)
            except:
                continue

        input_ids = []
        attention_masks = []
        token_type_ids = []

        for lyric in temp:
            try:
                input_id, attention_mask, token_type_id = electra_tokenizer(lyric, MAX_LEN)
                input_ids.append(input_id)
                attention_masks.append(attention_mask)
                token_type_ids.append(token_type_id)

            except Exception as e:
                print(e)
                pass
        prediction = self.model1.predict(input_ids, verbose=0)

        return prediction

    # Function for Preprocessing 5classes classification(angry, dislike, fear, sad, surprise)
    def predict_neg(self, lyrics):
        temp = []
        lyrics = self.preprocessing(lyrics)
        lyrics = self.slicing(lyrics)
        for lyric in lyrics:
            try:
                txt = ' '.join(lyric)
                temp.append(txt)
            except:
                continue

        input_ids = []
        attention_masks = []
        token_type_ids = []

        for lyric in temp:
            try:
                input_id, attention_mask, token_type_id = electra_tokenizer(lyric, MAX_LEN)
                input_ids.append(input_id)
                attention_masks.append(attention_mask)
                token_type_ids.append(token_type_id)

            except Exception as e:
                print(e)
                pass
        prediction = self.model2.predict(input_ids, verbose=0)

        return prediction

    def transform_neg(self, array):
        result = self.encoder2.inverse_transform([np.argmax(array)])
        return result

    def transform3classes(self, array):
        result = self.encoder1.inverse_transform([np.argmax(array)])
        return result

    def predictAll(self, lyrics):
        # 데이터 입력
        data = lyrics[['SONG_ID', 'SONG_TITLE', 'LYRICS']]
        song_ids = lyrics['SONG_ID']
        # 3분류
        pred = dict()
        for id in tqdm(song_ids):
            lyric = data[data['SONG_ID']==id]['LYRICS'].values[0]
            prediction = self.predict3classes(lyric)
            score = sum(prediction)/len(prediction)
            senti = self.transform3classes(score)
            pred[id] = (score, senti)
        data['score'] = ''
        data['senti'] = ''
        for id in song_ids:
            sc, se = pred[id]
            data.loc[data[data['SONG_ID']==id].index, 'score'] = str(sc)
            data.loc[data[data['SONG_ID']==id].index, 'senti'] = se
        # 5분류
        neg_data = data[data['senti']=='neg']
        pos_data = data[data['senti']=='pos']
        neg_ids = neg_data['SONG_ID']
        pos_ids = pos_data['SONG_ID']
        pos_pred = dict()
        neg_pred = dict()
        # 대분류가 neg인 경우
        for id in tqdm(neg_ids):
            lyric = data[data['SONG_ID']==id]['LYRICS'].values[0]
            prediction = self.predict_neg(lyric)
            score = sum(prediction)/len(prediction)
            senti = self.transform_neg(score)
            neg_pred[id] = (score, senti)
        data['final_score'] = ''
        data['final_senti'] = ''
        data['total'] = ''
        for key in neg_ids:
            sc, se = neg_pred[key]
            data.loc[data[data['SONG_ID']==key].index, 'final_score'] = str(sc)
            data.loc[data[data['SONG_ID']==key].index, 'final_senti'] = se
        # neg인 경우의 감정 지표 계산
        for key in neg_ids:
            neg_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'final_score'].tolist()[0][1:-1], dtype=float, sep=' ')
            pos_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'score'].tolist()[0][1:-1], dtype=float, sep=' ')
            ratio = np.array(((pos_sc[int(le.transform(['neg'])[0])]+pos_sc[1]*0.5), (pos_sc[int(le.transform(['pos'])[0])]+pos_sc[1]*0.5)))
            rated_sc = np.append(np.max(neg_sc)*(ratio[1]/ratio[0]), neg_sc)
            total_sc = rated_sc/(sum(rated_sc))
            data.loc[data[data['SONG_ID']==key].index, 'total'] = str(total_sc)
        # 대분류가 pos인 경우
        for id in tqdm(pos_ids):
            lyric = data[data['SONG_ID']==id]['LYRICS'].values[0]
            prediction = self.predict_neg(lyric)
            score = sum(prediction)/len(prediction)
            pos_pred[id] = (score, 'happy')
        for key in pos_ids:
            sc, se = pos_pred[key]
            data.loc[data[data['SONG_ID']==key].index, 'final_score'] = str(sc)
            data.loc[data[data['SONG_ID']==key].index, 'final_senti'] = se
        # pos인 경우의 감정 지표 계산
        for key in pos_ids:
            neg_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'final_score'].tolist()[0][1:-1], dtype=float, sep=' ')
            pos_sc = np.fromstring(data.loc[data[data['SONG_ID']==key].index, 'score'].tolist()[0][1:-1], dtype=float, sep=' ')
            ratio = np.array(((pos_sc[int(le.transform(['neg'])[0])]+pos_sc[1]*0.5), (pos_sc[int(le.transform(['pos'])[0])]+pos_sc[1]*0.5)))
            rated_sc = np.append(ratio[1], ratio[0]*neg_sc)
            total_sc = rated_sc/(sum(rated_sc))
            data.loc[data[data['SONG_ID']==key].index, 'total'] = str(total_sc)
        # 중립 제거
        data = data.drop(data[data['senti']=='neut'].index)
        return data

In [10]:
model = EmotionClassfier(le, neg_le)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

In [11]:
model.call()

In [12]:
test = pd.read_csv(DATA_IN_PATH+'test.csv')

In [13]:
model.predictAll(test)

100%|██████████| 20/20 [00:07<00:00,  2.56it/s]
c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\project_huiyeol\nlp\project\venv\lib\site-packages\ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\project_huiyeol\nlp\project\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,SONG_ID,SONG_TITLE,LYRICS,score,senti,final_score,final_senti,total
0,9270,매직 카펫 라이드,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...,[0.121995 0.293892 0.584113],pos,[0.148379 0.245044 0.067424 0.509761 0.029392],happy,[0.731059 0.039905 0.065902 0.018133 0.137096 ...
1,19807,벌써 일년,처음이라 그래 며칠뒤엔 \n괜찮아져 \n그 생각만으로 벌써 일년이 \n너와 만든 기...,[0.319238 0.256552 0.424209],pos,[0.167217 0.091865 0.019813 0.712458 0.008647],happy,[0.552486 0.074832 0.041111 0.008867 0.318835 ...
2,32616,3!4!,여기 숨쉬는 이 시간은\n나를 어데로 데려갈까\n많은 기쁨과 한숨들이\n뒤섞인 이곳...,[0.069013 0.425081 0.505906],pos,[0.054421 0.044274 0.028501 0.866249 0.006555],happy,[0.718446 0.015322 0.012465 0.008025 0.243895 ...
3,33133,DOC와 춤을,젓가락질 잘해야만 밥을 먹나요 \n잘못해도 서툴러도 밥 잘 먹어요 \n그러나 주위사...,[0.38072 0.380664 0.238616],neg,[0.138274 0.582804 0.094649 0.175393 0.00888 ],dislike,[0.304481 0.096172 0.405351 0.06583 0.121989 ...
4,70696,취중진담(醉中眞談),그래 난 취했는지도 몰라 실수인지도 몰라 \n아침이면 까마득히 생각이 안나 \n불안...,[0.519385 0.197966 0.282649],neg,[0.062054 0.205801 0.140273 0.587172 0.0047 ],sad,[0.26599 0.045548 0.15106 0.102962 0.43099 ...
5,81013,커플,예전보다 지금 니가\n더욱 괜찮을거야 \n허전했던 나의 빈 곳을\n이젠 채워 줬으니...,[0.141428 0.213228 0.645344],pos,[0.104817 0.078981 0.04542 0.742258 0.028525],happy,[0.751958 0.025999 0.019591 0.011266 0.184111 ...
6,85257,바꿔,모두 제정신이 아니야 다들 \n미쳐가고만 있어 \n어느 누굴 믿어 어찌믿어 \...,[0.450147 0.353492 0.196361],neg,[0.22911 0.256968 0.035553 0.454508 0.023862],sad,[0.212914 0.180329 0.202256 0.027983 0.357737 ...
7,85258,와,사실이 아니길 믿고 싶었어 \n널 놓치기 싫었어 \n혹시나 우리의 사랑이 잘못 돼\...,[0.622023 0.177687 0.20029 ],neg,[0.075827 0.162271 0.031417 0.728991 0.001494],sad,[0.228696 0.058486 0.12516 0.024232 0.562274 ...
8,87048,땡벌,아 당신은 못믿을 사람\n아 당신은 철없는 사람\n아무리 달래봐도\n어쩔순 없지만\...,[0.78938 0.114164 0.096456],neg,[0.093934 0.392661 0.029438 0.482071 0.001896],sad,[0.080411 0.086381 0.361087 0.027071 0.443307 ...
9,105014,제주도 푸른밤,떠나요 둘이서 모든 것 훌훌 버리고 \n제주도 푸른밤 그 별 아래 \n\n이제는 더...,[0.178256 0.241083 0.580661],pos,[0.067633 0.283215 0.029714 0.617043 0.002395],happy,[0.701202 0.020209 0.084624 0.008878 0.184371 ...
